In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 71.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=eadf092f8fa9b9a03600aa9c6caef47aea32fcb224c5d1fc91329685f2ad39d6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
# загружаем библиотеки
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark_connection
spark = SparkSession(sc) # open spark session

## Задание. Обучите модель классификации для цветков Iris'а

---
Примерная последовательность действий:
1. Взять данные iris.csv
2. Загрузить в pyspark
3. При помощи VectorAssembler преобразовать все колонки с признаками в одну
(использовать PipeLine)
4. Разбить данные на train и test
5. Создать модель логистической регресии и обучить ее
6. Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test
множестве

---

### 1-2. Загружаем iris через pyspark

In [6]:
# Используем options() чтобы задать несколько опций:
# header=True - Для чтения использует первую строку в качестве имен столбцов
# sep="," - Устанавливает разделитель для каждого поля и значения
# quote="" - Нужно установить пустую строку для отключения кавычки, тк quote по-умолчанию ", тк ковычки не являются разделителем
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')

In [7]:
df.show(5)

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
+-------------+---------------+----------------+---------------+------------+
only showing top 5 rows



In [8]:
df.printSchema()

root
 |-- "sepal.length: string (nullable = true)
 |-- ""sepal.width"": string (nullable = true)
 |-- ""petal.length"": string (nullable = true)
 |-- ""petal.width"": string (nullable = true)
 |-- ""variety""": string (nullable = true)



In [9]:
#преобразуем названия колонок, создав новый датафрейм
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)

#удалим двойные ковычки в первом и последнем атрибутах
df_new = df_new.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ""))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ""))

#преобразуем в другие типы данных
df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [10]:
df_new.show(5) #посмотрим на преобразование

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [11]:
df_new.printSchema() #посмотрим на типы атрибутов

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [12]:
df_new.summary().show()
#df_new.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|     null|
|    50%|               5.8|                3.0|               4.3|               1.3|     null|
|    75%|               6.4|                3.3|               5.1|               1.8|     null|
|    max|               7.9|  

### 3. При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать PipeLine)

**Векторизация признаков**

Поскольку алгоритмы машинного обучения в PySpark принимают на вход только вектора, то нужно  провести векторизацию. Для преобразования признаков в вектора используется класс VectorAssembler. Объект этого класса принимает в качестве аргументов список с названиями признаков, которые нужно векторизовать (inputCols), и название преобразованного признака (outputCol). После создания объекта VectorAssembler вызывается метод transform.

- Признаки, по которым будет определяться вид: sepal_length, sepal_width, petal_length, petal_width

- Тк признак variety является строковым, а алгоритмы Machine Learning в PySpark работают с числовым значениями, то преобразуем значения столбца variety в числовой вид с помощью StringIndexer: https://spark.apache.org/docs/latest/ml-features#stringindexer

Чтобы указаные шаги делать не один раз, можно их объединить в Pipeline преобразования и применять как единую операцию: https://spark.apache.org/docs/latest/ml-pipeline.html

In [13]:
pipeline = Pipeline(stages = 
[
  StringIndexer(inputCol='variety', outputCol='varietyInd'),
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')
]
)

In [14]:
# подгоняем через заданный pipeline наш датасет
pipelineTrained = pipeline.fit(df_new)

In [15]:
# применяем метод transform для объекта VectorAssembler (преобразуем признаки в вектор) 
pipelineTrained.transform(df_new).show()

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

In [16]:
# с новым признаком Features создаем новый датафрейм
df_features = pipelineTrained.transform(df_new)

### 4. Разбить данные на train и test

Для решения задач Machine Learning всегда нужно иметь, как минимум, две выборки — обучающую и тестовую. На обучающей мы будем обучать модель, а на тестовой проверять эффективность обученной модели. В PySpark сделать это очень просто, нужно просто вызвать метод randomSplit, который разделит исходный датасет в заданной пропорции. Мы разделим в пропорции 80:20.

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html

In [17]:
# назначаем соответствующие веса и указываем seed (рандомное разделение ввыборки сохранилось)
train, test = df_features.randomSplit([0.8, 0.2], seed=12345)

In [18]:
# посмотрим на обучающую выборку
train.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|    Setosa|       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.4|         1.4|        0.3|    Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         4.6|        3.6|         1.0|        0.2|    Setosa|       0.0|[4.6,3.

In [20]:
# посмотрим на тестовую выборку
test.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|
|         5.1|        3.5|         1.4|        0.3|    Setosa|       0.0|[5.1,3.5,1.4,0.3]|
|         5.4|        3.4|         1.5|        0.4|    Setosa|       0.0|[5.4,3.4,1.5,0.4]|
|         5.4|        3.9|         1.3|        0.4|    Setosa|       0.0|[5.4,3.9,1.3,0.4]|
|         5.7|        2.8|         4.1|        1.3|Versicolor|       1.0|[5.7,2.

### 5. Создать модель логистической регресии и обучить ее

Теперь воспользуемся логистической регрессией (Logistic Regression), которая есть в PySpark, в качестве алгоритма Machine learning. Для этого нужно указать признаки, на которых модель обучается, и признак, который нужно классифицировать. Мы преобразовали признаки цветка в вектор под названием features, поэтому именно его и указываем в аргументе, а целевым признаком будет числовое поле varietyInd

In [21]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

Осталось только получить предсказания. Для этого вызывается метод transform, который принимает тестовую выборку, а также обучающую (как обучилась модель):

In [22]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [23]:
# посмотрим, какие предсказания дает модель на обучающей выборке
train_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|[67.7027919587531...|[1.0,1.6424718354...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|[57.0087434717148...|[1.0,1.4036227809...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|[61.7475610236625...|[1.0,2.4772179810...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|[68.8060989728479...|[1.0,1.2455403381.

In [24]:
# посмотрим, какие предсказания дает модель на тестовой выборке
test_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|[65.8143591752207...|[1.0,8.9932431201...|       0.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|[52.7723416307807...|[1.0,9.3248066295...|       0.0|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|[64.6690738893863...|[1.0,1.7992914773...|       0.0|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|[71.4022940655650...|[1.0,5.6805112138.

### 6. Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве

Для оценки качества предсказания в spark реализованно несколько классов Если мы решаем задачу бинарной классификации (то есть классов - 2), то нам подойдет BinaryCLassificationEvaluator, а если классов больше 2-х, то MulticlassClassificationEvaluator.

Проверим эффективность модели, используя метрику качества MulticlassClassificationEvaluator, тк признак variety состоит из 3х значений. Нужно лишь указать целевой признак ***varietyInd***, а затем вызвать метод ***evaluate*** и передать в него наши предсказания

In [25]:
ev = MulticlassClassificationEvaluator(labelCol='varietyInd')

In [26]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%')

Точность предсказания на обучающей выборке: 98.44961240310077 %


In [27]:
print('Точность предсказания на тестовой выборке:', ev.evaluate(test_res)*100, '%')

Точность предсказания на тестовой выборке: 100.0 %


### Выводы:
Как мы видим, на обучающей выборке мы обучили модель с точностью до 98,4%, что на тестовой выборке показало нам 100%, т.е. идеальный вариант.